# Задание 2: Работа с датасетами (30 баллов)

Извиняюсь ,что не импортировал,у меня почему-то VS не видело файл из первой части

In [151]:
import torch
import torch.nn as nn

class LinearModel(nn.Module):
    def __init__(self,in_features,out_features):
        super().__init__()
        self.layer1 = nn.Linear(in_features,out_features)
    
    def forward(self,x):
        return self.layer1(x)


#l1_loss 
def l1_loss(output, target, activations, criterion, l1_lambda):
    # Вычисляем основную потерю
    main_loss = criterion(output, target)
    
    # Убедимся, что main_loss - скаляр
    if main_loss.dim() > 0:
        main_loss = main_loss.mean()  # Усредняем, если это не скаляр
    
    # Вычисляем L1-регуляризацию
    l1_penalty = torch.norm(activations, p=1)
    
    # Убедимся, что l1_penalty - скаляр
    if l1_penalty.dim() > 0:
        l1_penalty = l1_penalty.mean()
    
    return main_loss + l1_lambda * l1_penalty


#класс для ранней остановки
class EarlyStopping:
    """"
    patience-количество эпох, которые нужно подождать, прежде чем остановиться, если улучшения не будет.
    delta-Minimum change in the monitored quantity to qualify as an improvement.
    best_score,best_model_state -Track the best validation score and model state.

    
    """
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.best_model_state = None

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model_state = model.state_dict()
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_state = model.state_dict()
            self.counter = 0

    def load_best_model(self, model):
        model.load_state_dict(self.best_model_state)

In [169]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
import os

class ClassificationMetrics:
    def __init__(self):
        """
        Инициализация класса метрик классификации
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def to_one_hot(self, labels, num_classes):
        """
        Преобразование меток в one-hot encoding
        
        Args:
            labels: тензор меток (batch_size,)
            num_classes: количество классов
            
        Returns:
            one-hot тензор (batch_size, num_classes)
        """
        return torch.nn.functional.one_hot(labels, num_classes).float()

    def precision(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики Precision
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            precision score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        
        if average == 'micro':
            true_positives = torch.sum(y_true == y_pred).float()
            predicted_positives = len(y_pred)
            return true_positives / predicted_positives if predicted_positives > 0 else 0
        
        precision_per_class = []
        weights = []
        
        for c in range(num_classes):
            true_positives = torch.sum((y_true == c) & (y_pred == c)).float()
            predicted_positives = torch.sum(y_pred == c).float()
            
            precision = true_positives / predicted_positives if predicted_positives > 0 else 0
            precision_per_class.append(precision)
            
            if average == 'weighted':
                weights.append(torch.sum(y_true == c).float() / len(y_true))
        
        precision_per_class = torch.tensor(precision_per_class)
        
        if average == 'macro':
            return torch.mean(precision_per_class).item()
        elif average == 'weighted':
            return torch.sum(precision_per_class * torch.tensor(weights)).item()
        
        return precision_per_class.tolist()

    def recall(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики Recall
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            recall score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        
        if average == 'micro':
            true_positives = torch.sum(y_true == y_pred).float()
            actual_positives = len(y_true)
            return true_positives / actual_positives if actual_positives > 0 else 0
        
        recall_per_class = []
        weights = []
        
        for c in range(num_classes):
            true_positives = torch.sum((y_true == c) & (y_pred == c)).float()
            actual_positives = torch.sum(y_true == c).float()
            
            recall = true_positives / actual_positives if actual_positives > 0 else 0
            recall_per_class.append(recall)
            
            if average == 'weighted':
                weights.append(torch.sum(y_true == c).float() / len(y_true))
        
        recall_per_class = torch.tensor(recall_per_class)
        
        if average == 'macro':
            return torch.mean(recall_per_class).item()
        elif average == 'weighted':
            return torch.sum(recall_per_class * torch.tensor(weights)).item()
        
        return recall_per_class.tolist()

    def f1_score(self, y_true, y_pred, average='macro'):
        """
        Вычисление метрики F1-score
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            average: тип усреднения ('macro', 'micro', 'weighted')
            
        Returns:
            f1 score
        """
        precision = self.precision(y_true, y_pred, average)
        recall = self.recall(y_true, y_pred, average)
        
        if average in ['macro', 'weighted']:
            return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        elif average == 'micro':
            return precision
        
        f1_scores = []
        for p, r in zip(precision, recall):
            f1 = 2 * (p * r) / (p + r) if (p + r) > 0 else 0
            f1_scores.append(f1)
        return f1_scores

    def roc_auc(self, y_true, y_scores, multi_class='ovr'):
        """
        Вычисление метрики ROC-AUC
        
        Args:
            y_true: истинные метки (batch_size,)
            y_scores: вероятности предсказаний (batch_size, num_classes)
            multi_class: 'ovr' (one-vs-rest) или 'ovo' (one-vs-one)
            
        Returns:
            roc-auc score
        """
        y_true = torch.tensor(y_true, dtype=torch.long).cpu().numpy()
        y_scores = torch.tensor(y_scores, dtype=torch.float).cpu().numpy()
        
        num_classes = y_scores.shape[1]
        y_true_one_hot = self.to_one_hot(torch.tensor(y_true), num_classes).cpu().numpy()
        
        auc_scores = []
        
        if multi_class == 'ovr':
            for i in range(num_classes):
                fpr, tpr, _ = roc_curve(y_true_one_hot[:, i], y_scores[:, i])
                auc_score = auc(fpr, tpr)
                auc_scores.append(auc_score)
            return np.mean(auc_scores)
        
        elif multi_class == 'ovo':
            auc_sum = 0
            n_pairs = 0
            for i in range(num_classes):
                for j in range(i+1, num_classes):
                    mask = np.logical_or(y_true == i, y_true == j)
                    y_true_binary = (y_true[mask] == i).astype(int)
                    y_scores_binary = y_scores[mask, i] / (y_scores[mask, i] + y_scores[mask, j])
                    fpr, tpr, _ = roc_curve(y_true_binary, y_scores_binary)
                    auc_sum += auc(fpr, tpr)
                    n_pairs += 1
            return auc_sum / n_pairs if n_pairs > 0 else 0
        
        return auc_scores

    def confusion_matrix(self, y_true, y_pred, plot=True, save_path=None):
        """
        Вычисление и визуализация confusion matrix с возможностью сохранения
        
        Args:
            y_true: истинные метки (batch_size,)
            y_pred: предсказанные метки (batch_size,)
            plot: флаг для отображения визуализации
            save_path: путь для сохранения графика (например, 'confusion_matrix.png')
            
        Returns:
            confusion matrix
        """
        y_true = torch.tensor(y_true, dtype=torch.long).to(self.device)
        y_pred = torch.tensor(y_pred, dtype=torch.long).to(self.device)
        
        num_classes = len(torch.unique(y_true))
        cm = torch.zeros((num_classes, num_classes), dtype=torch.long)
        
        for t, p in zip(y_true, y_pred):
            cm[t, p] += 1
        
        if plot:
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm.numpy(), annot=True, fmt='d', cmap='Blues')
            plt.title('Confusion Matrix')
            plt.ylabel('True Label')
            plt.xlabel('Predicted Label')
            
            if save_path:
                # Создаём директорию, если она не существует
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                plt.savefig(save_path, dpi=300, bbox_inches='tight')
                print(f"Confusion matrix сохранена в {save_path}")
            
            plt.show()
            plt.close()
        
        return cm


Создайте кастомный класс датасета для работы с CSV файлами:
 - Загрузка данных из файла
 - Предобработка (нормализация, кодирование категорий)
 - Поддержка различных форматов данных (категориальные, числовые, бинарные и т.д.)

2.2 Эксперименты с различными датасетами (15 баллов)
<br>Найдите csv датасеты для регрессии и бинарной классификации и, применяя наработки из предыдущей части задания, обучите линейную и логистическую регрессию
<br>

Для регрессии возьмем датасет Prediction of Insurance Charges
его колонки:
 -age
 -sex
 -bmi: индекс массы тела
 -Children: количество детей
 -Smoker: курящий/некурящий
 -Region: регион жизни
 -Charges(target):цена страховки

In [161]:
from torch.utils.data import Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import numpy as np
import torch

class CustomDataset(Dataset):
    def __init__(self, path_file, numeric_columns: list, string_columns: list, binary_columns: list, target_column: str):
        self.path_file = path_file
        self.numeric_columns = numeric_columns
        self.string_columns = string_columns
        self.binary_columns = binary_columns
        self.target_column = target_column
        self.label_encoders = {}
        self.one_hot_encoders = {}

        self.df = pd.read_csv(path_file)
        self.df = self.df.dropna()

        self._convert_numeric()
        self._convert_string()
        self._convert_binary()

    def _convert_numeric(self):
        if self.numeric_columns:
            scaler = StandardScaler()
            self.df[self.numeric_columns] = scaler.fit_transform(self.df[self.numeric_columns])

    def _convert_string(self):
        if self.string_columns:
            for column in self.string_columns:
                le = LabelEncoder()
                self.df[column] = self.df[column].fillna('missing')
                self.df[column] = le.fit_transform(self.df[column])
                self.label_encoders[column] = le

    def _convert_binary(self):
        if self.binary_columns:
            for column in self.binary_columns:
                encoder = OneHotEncoder(sparse_output=False, drop='first')
                transformed = encoder.fit_transform(self.df[[column]])
                new_columns = [f"{column}_{cat}" for cat in encoder.categories_[0][1:]]
                self.df[new_columns] = transformed
                self.df = self.df.drop(column, axis=1)
                self.one_hot_encoders[column] = encoder

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        data_one_row = self.df.iloc[index]
        train = data_one_row.drop(self.target_column).values.astype(np.float32)  # Преобразуем в numpy float32
        target = np.array(data_one_row[self.target_column], dtype=np.float32)  # Преобразуем в numpy float32
        return torch.tensor(train), torch.tensor(target)  # Возвращаем тензоры

Обучение регрессии

In [156]:
from torch.utils.data import DataLoader,SubsetRandomSampler
from sklearn.model_selection import train_test_split

import torch.optim as optim
dataset = CustomDataset(path_file="data/insurance.csv",numeric_columns=["age","bmi"],string_columns=["region"],binary_columns=["sex","smoker"],target_column="charges")



batch_size: int = 32,
num_epochs: int = 100,
learning_rate: float = 0.001,
l1_lambda: float = 0.01,
patience: int = 5,
validation_split: float = 0.2


in_features = len(dataset.df.columns) - 1  # Все колонки, кроме целевой
out_features = 1 


model = LinearModel(in_features, out_features)
criterion = nn.MSELoss()  # Функция потерь для регрессии
optimizer = torch.optim.Adam(params =model.parameters(), lr=0.001)

    # Разделяем данные на обучающую и валидационную выборки
dataset_size = len(dataset)
indices = list(range(dataset_size))
train_indices, val_indices = train_test_split(
        indices, test_size=validation_split, random_state=42
    )

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

    # Инициализируем раннюю остановку
early_stopping = EarlyStopping(patience=patience, delta=0)

    # Обучение
print("Starting training...")
for epoch in range(100):
        # Обучающий режим
        model.train()
        train_loss = 0.0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to( dtype=torch.float32), target.to(dtype=torch.float32)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.view(-1, 1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Валидация
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to( dtype=torch.float32), target.to( dtype=torch.float32)
                output = model(data)
                loss = criterion(output, target.view(-1, 1))  # Только MSE для валидации
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

       

   

Starting training...
Epoch 1/(100,), Train Loss: 327476651.764706, Val Loss: 311880878.222222
Epoch 2/(100,), Train Loss: 319082727.529412, Val Loss: 330084360.888889
Epoch 3/(100,), Train Loss: 318165669.176471, Val Loss: 308756250.666667
Epoch 4/(100,), Train Loss: 320297941.176471, Val Loss: 340686343.111111
Epoch 5/(100,), Train Loss: 316716095.529412, Val Loss: 327233368.888889
Epoch 6/(100,), Train Loss: 315532083.764706, Val Loss: 311472371.555556
Epoch 7/(100,), Train Loss: 313731534.588235, Val Loss: 313265361.777778
Epoch 8/(100,), Train Loss: 313639895.058824, Val Loss: 307650261.333333
Epoch 9/(100,), Train Loss: 314554938.823529, Val Loss: 306559799.111111
Epoch 10/(100,), Train Loss: 313533564.705882, Val Loss: 326352001.777778
Epoch 11/(100,), Train Loss: 319079892.705882, Val Loss: 301189192.000000
Epoch 12/(100,), Train Loss: 314771250.352941, Val Loss: 303986839.111111
Epoch 13/(100,), Train Loss: 315511492.705882, Val Loss: 327465466.666667
Epoch 14/(100,), Train Los

Датасет Титаник

<h3>Описание названия колонок:</h3>
<ul>
<li>survival - Выживание (0 = Нет; 1 = Да)</li>
<li>pclass - Класс пассажира (1 = 1-й; 2 = 2-й; 3 = 3-й)</li>
<li>name - Имя</li>
<li>sex - Пол</li>
<li>age - Возраст</li>
<li>sibsp - Количество братьев/сестер или супругов на борту</li>
<li>parch - Количество родителей/детей на борту</li>
<li>ticket - Номер билета</li>
<li>fare - Стоимость билета пассажира</li>
<li>cabin - Каюта</li>
<li>embarked - Порт посадки (C = Шербур; Q = Квинстаун; S = Саутгемптон)</li>
</ul>

In [168]:
from torch.utils.data import DataLoader,SubsetRandomSampler
from sklearn.model_selection import train_test_split

import torch.optim as optim
dataset = CustomDataset(path_file="data/Titanic-Dataset.csv",numeric_columns=["Age"],string_columns=["Name","Ticket","Embarked","Cabin"]
                        ,binary_columns=["Sex"]
                        ,target_column="Survived")



batch_size: int = 32,
num_epochs: int = 100,
learning_rate = 0.001,
l1_lambda: float = 0.01,
patience: int = 5,
validation_split: float = 0.2
lr = 0.001

in_features = len(dataset.df.columns) - 1  # Все колонки, кроме целевой
out_features = 1 


model = LinearModel(in_features, out_features)
criterion = nn.BCEWithLogitsLoss()  # Функция потерь для регрессии
optimizer = torch.optim.Adam(params =model.parameters(), lr=lr)

    # Разделяем данные на обучающую и валидационную выборки
dataset_size = len(dataset)
indices = list(range(dataset_size))
train_indices, val_indices = train_test_split(
        indices, test_size=validation_split, random_state=42
    )

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

    # Инициализируем раннюю остановку
early_stopping = EarlyStopping(patience=patience, delta=0)

    # Обучение
print("Starting training...")
for epoch in range(100):
        # Обучающий режим
        model.train()
        train_loss = 0.0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to( dtype=torch.float32), target.to(dtype=torch.float32)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.view(-1, 1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Валидация
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to( dtype=torch.float32), target.to( dtype=torch.float32)
                output = model(data)
                loss = criterion(output, target.view(-1, 1))  # Только MSE для валидации
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

       

   

Starting training...
Epoch 1/(100,), Train Loss: 76.208521, Val Loss: 61.642212
Epoch 2/(100,), Train Loss: 72.596861, Val Loss: 86.661995
Epoch 3/(100,), Train Loss: 70.062453, Val Loss: 43.922116
Epoch 4/(100,), Train Loss: 69.319249, Val Loss: 61.893475
Epoch 5/(100,), Train Loss: 64.622758, Val Loss: 80.970697
Epoch 6/(100,), Train Loss: 64.092899, Val Loss: 77.216120
Epoch 7/(100,), Train Loss: 60.315874, Val Loss: 61.351910
Epoch 8/(100,), Train Loss: 58.908494, Val Loss: 44.223593
Epoch 9/(100,), Train Loss: 55.788217, Val Loss: 68.497637
Epoch 10/(100,), Train Loss: 52.496656, Val Loss: 60.455069
Epoch 11/(100,), Train Loss: 49.609174, Val Loss: 68.109592
Epoch 12/(100,), Train Loss: 49.171133, Val Loss: 52.961517
Epoch 13/(100,), Train Loss: 47.913427, Val Loss: 45.494944
Epoch 14/(100,), Train Loss: 44.276793, Val Loss: 36.695910
Epoch 15/(100,), Train Loss: 42.724540, Val Loss: 31.425717
Epoch 16/(100,), Train Loss: 43.076008, Val Loss: 42.778643
Epoch 17/(100,), Train Loss: